# Kepler Framework, Emulate v9

In [7]:
from kepler.pandas.menu       import ElectronSequence as Chain
from kepler.pandas.readers    import load, load_in_loop, load_hdf
from core import Efficiency
from Gaugi import mkdir_p, progressbar
import numpy as np
import pandas as pd
import collections
import os
from pprint import pprint
from copy import deepcopy

import gc
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
output_path = 'output/emulation'
mkdir_p(output_path)

In [9]:
dpath_data17 = '../../data/output/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'
dpath_data18 = '../../data/output/data18_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'
dpath_mc16   = '../../data/output/mc16_13TeV.302236_309995_341330.sgn.boosted_probes.WZ_llqq_plus_radion_ZZ_llqq_plus_ggH3000.merge.dataframe.h5'

In [10]:
tpath_data17 = 'output/emulation/data17_table_v9.h5'
tpath_data18 = 'output/emulation/data18_table_v9.h5'
tpath_mc16   = 'output/emulation/mc16_table_v9.h5'

## Setup Chains:

In [11]:
# create my chain
chains = [
            Chain( "HLT_e17_lhvloose_nod0_ringer_v9_L1EM15VHI", L1Seed = 'L1_EM15VHI' , l2calo_column = 'ringer_v9_vloose'),
            Chain( "HLT_e28_lhtight_nod0_ringer_v9_ivarloose", L1Seed = 'L1_EM22VHI' , l2calo_column = 'ringer_v9_tight' ),
            Chain( "HLT_e60_lhmedium_nod0_ringer_v9_L1EM24VHI", L1Seed = 'L1_EM24VHI' , l2calo_column = 'ringer_v9_medium'),
            Chain( "HLT_e140_lhloose_nod0_ringer_v9" , L1Seed = 'L1_EM24VHI', l2calo_column = 'ringer_v9_loose'),
]

def emulate( df ):
    for chain in progressbar(chains, prefix='Emulate...'):
        chain.apply(df)

OrderedDict([('L1Seed', 'L1_EM15VHI'),
             ('L2Calo', (14000.0, 'ringer_v9_vloose')),
             ('L2', 'trig_L2_el_cut_pt15to20'),
             ('EFCalo', 17000.0),
             ('HLT', (17000.0, 'trig_EF_el_lhvloose'))])
OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (25000.0, 'ringer_v9_tight')),
             ('L2', 'trig_L2_el_cut_pt20to50'),
             ('EFCalo', 28000.0),
             ('HLT', (28000.0, 'trig_EF_el_lhtight_ivarloose'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (57000.0, 'ringer_v9_medium')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 60000.0),
             ('HLT', (60000.0, 'trig_EF_el_lhmedium'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (137000.0, 'ringer_v9_loose')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 140000.0),
             ('HLT', (140000.0, 'trig_EF_el_lhloose'))])


## Emulate 2017:

Take something like 1 hour

In [12]:
data17_df = pd.concat( (load_hdf(dpath_data17), load_hdf(tpath_data17)), axis=1)

In [13]:
emulate(data17_df)

Emulate...: 100%|██████████| 4/4 [01:28<00:00, 22.10s/it]


In [14]:
data17_df.shape

(43311283, 111)

### Validate:

In [15]:
eff_data17_Zee = Efficiency( output_path+'/data17_Zee_efficiency_v9.root')

2022-03-13 10:04:48,762 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data17_Zee_efficiency_v9.root


In [16]:
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v9_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v9_ivarloose' , pidname='el_lhtight' , reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v9_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v9'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [01:59<00:00, 23.92s/it]


HLT_e17_lhvloose_nod0_ringer_v9_L1EM15VHI
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.1069  | 30158981/32047563 |
| L2Calo | 93.5023  | 29965209/32047563 |
|   L2   | 93.2645  | 29889005/32047563 |
| EFCalo | 91.9537  | 29468909/32047563 |
|  HLT   | 90.0304  | 28852564/32047563 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [01:26<00:00, 17.27s/it]


HLT_e28_lhtight_nod0_ringer_v9_ivarloose
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.6937  | 21557594/22765602 |
| L2Calo | 94.2517  | 21456963/22765602 |
|   L2   | 94.1373  | 21430918/22765602 |
| EFCalo | 93.2128  | 21220458/22765602 |
|  HLT   | 87.0395  | 19815055/22765602 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [00:06<00:00,  1.25s/it]


HLT_e60_lhmedium_nod0_ringer_v9_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.5079  | 1361055/1367786 |
| L2Calo | 97.2273  | 1329861/1367786 |
|   L2   | 97.1376  | 1328635/1367786 |
| EFCalo | 95.0474  | 1300045/1367786 |
|  HLT   | 91.7724  | 1255250/1367786 |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 18.42it/s]

HLT_e140_lhloose_nod0_ringer_v9
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 100.0000 | 60541/60541  |
| L2Calo | 97.8510  | 59240/60541  |
|   L2   | 97.7817  | 59198/60541  |
| EFCalo | 96.4685  | 58403/60541  |
|  HLT   | 94.5607  | 57248/60541  |
+--------+----------+--------------+


In [17]:
eff_data17_Zee.save()

In [18]:
eff_data17_JF17 = Efficiency( output_path+'/data17_JF17_efficiency_v9.root')

2022-03-13 10:09:14,568 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data17_JF17_efficiency_v9.root


In [19]:
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e17_lhvloose_nod0_ringer_v9_L1EM15VHI', pidname='!el_lhvloose', reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e28_lhtight_nod0_ringer_v9_ivarloose' , pidname='!el_lhvloose', reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e60_lhmedium_nod0_ringer_v9_L1EM24VHI', pidname='!el_lhvloose', reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e140_lhloose_nod0_ringer_v9'          , pidname='!el_lhvloose', reset=True)

Filling...: 100%|██████████| 5/5 [00:22<00:00,  4.54s/it]


HLT_e17_lhvloose_nod0_ringer_v9_L1EM15VHI
+--------+----------+------------------+
|  Step  | Eff [%%] |   passed/total   |
+--------+----------+------------------+
| L1Calo | 84.4275  | 8738241/10349992 |
| L2Calo |  1.6708  | 172928/10349992  |
|   L2   |  1.5943  | 165011/10349992  |
| EFCalo |  1.4242  | 147406/10349992  |
|  HLT   |  0.1416  |  14658/10349992  |
+--------+----------+------------------+


Filling...: 100%|██████████| 5/5 [00:20<00:00,  4.06s/it]


HLT_e28_lhtight_nod0_ringer_v9_ivarloose
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 90.6060  | 8384948/9254300 |
| L2Calo |  0.5624  |  52044/9254300  |
|   L2   |  0.5240  |  48496/9254300  |
| EFCalo |  0.4206  |  38924/9254300  |
|  HLT   |  0.0015  |   137/9254300   |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:17<00:00,  3.43s/it]


HLT_e60_lhmedium_nod0_ringer_v9_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9012  | 7844422/7852181 |
| L2Calo |  0.5304  |  41647/7852181  |
|   L2   |  0.4845  |  38044/7852181  |
| EFCalo |  0.3575  |  28074/7852181  |
|  HLT   |  0.0018  |   139/7852181   |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:12<00:00,  2.56s/it]

HLT_e140_lhloose_nod0_ringer_v9
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9227  | 5629684/5634041 |
| L2Calo |  1.4126  |  79588/5634041  |
|   L2   |  1.3309  |  74982/5634041  |
| EFCalo |  1.0973  |  61825/5634041  |
|  HLT   |  0.0391  |   2204/5634041  |
+--------+----------+-----------------+


In [20]:
eff_data17_JF17.save()

In [21]:
del data17_df

## Emulate 2018:

In [22]:
data18_df = pd.concat( (load_hdf(dpath_data18), load_hdf(tpath_data18)), axis=1)

In [23]:
emulate(data18_df)

Emulate...: 100%|██████████| 4/4 [01:27<00:00, 21.98s/it]


In [24]:
data18_df.shape

(41873973, 111)

In [25]:
eff_data18_Zee = Efficiency( output_path+'/data18_Zee_efficiency_v9.root')

2022-03-13 10:14:48,507 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data18_Zee_efficiency_v9.root


In [26]:
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v9_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v9_ivarloose' , pidname='el_lhtight' , reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v9_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v9'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [01:59<00:00, 23.96s/it]


HLT_e17_lhvloose_nod0_ringer_v9_L1EM15VHI
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.1301  | 30705693/32620495 |
| L2Calo | 93.5456  | 30515029/32620495 |
|   L2   | 93.3144  | 30439624/32620495 |
| EFCalo | 91.9807  | 30004559/32620495 |
|  HLT   | 90.5449  | 29536184/32620495 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [01:26<00:00, 17.23s/it]


HLT_e28_lhtight_nod0_ringer_v9_ivarloose
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.6555  | 22067825/23313826 |
| L2Calo | 94.2127  | 21964577/23313826 |
|   L2   | 94.1175  | 21942386/23313826 |
| EFCalo | 93.3183  | 21756067/23313826 |
|  HLT   | 87.6039  | 20423816/23313826 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [00:06<00:00,  1.24s/it]


HLT_e60_lhmedium_nod0_ringer_v9_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.5563  | 1375545/1381675 |
| L2Calo | 97.2007  | 1342998/1381675 |
|   L2   | 97.1128  | 1341783/1381675 |
| EFCalo | 95.0239  | 1312921/1381675 |
|  HLT   | 92.0477  | 1271800/1381675 |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 18.75it/s]

HLT_e140_lhloose_nod0_ringer_v9
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9934  | 60385/60389  |
| L2Calo | 97.9632  | 59159/60389  |
|   L2   | 97.8655  | 59100/60389  |
| EFCalo | 96.6848  | 58387/60389  |
|  HLT   | 94.8799  | 57297/60389  |
+--------+----------+--------------+


In [27]:
eff_data18_Zee.save()

In [28]:
eff_data18_JF17 = Efficiency( output_path+'/data18_JF17_efficiency_v9.root')

2022-03-13 10:19:16,336 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data18_JF17_efficiency_v9.root


In [29]:
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e17_lhvloose_nod0_ringer_v9_L1EM15VHI', pidname='!el_lhvloose', reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e28_lhtight_nod0_ringer_v9_ivarloose' , pidname='!el_lhvloose', reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e60_lhmedium_nod0_ringer_v9_L1EM24VHI', pidname='!el_lhvloose', reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e140_lhloose_nod0_ringer_v9'          , pidname='!el_lhvloose', reset=True)

Filling...: 100%|██████████| 5/5 [00:18<00:00,  3.66s/it]


HLT_e17_lhvloose_nod0_ringer_v9_L1EM15VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 85.5183  | 7240922/8467098 |
| L2Calo |  2.0981  |  177644/8467098 |
|   L2   |  2.0191  |  170959/8467098 |
| EFCalo |  1.8527  |  156868/8467098 |
|  HLT   |  0.6132  |  51918/8467098  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:16<00:00,  3.31s/it]


HLT_e28_lhtight_nod0_ringer_v9_ivarloose
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 91.4008  | 6972786/7628804 |
| L2Calo |  0.8688  |  66281/7628804  |
|   L2   |  0.8329  |  63537/7628804  |
| EFCalo |  0.7337  |  55970/7628804  |
|  HLT   |  0.0404  |   3083/7628804  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:14<00:00,  2.84s/it]


HLT_e60_lhmedium_nod0_ringer_v9_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.8995  | 6555389/6561985 |
| L2Calo |  0.8187  |  53720/6561985  |
|   L2   |  0.7760  |  50921/6561985  |
| EFCalo |  0.6559  |  43037/6561985  |
|  HLT   |  0.0482  |   3163/6561985  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:10<00:00,  2.15s/it]

HLT_e140_lhloose_nod0_ringer_v9
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9159  | 4772502/4776518 |
| L2Calo |  1.8955  |  90541/4776518  |
|   L2   |  1.8126  |  86577/4776518  |
| EFCalo |  1.5898  |  75937/4776518  |
|  HLT   |  0.3245  |  15502/4776518  |
+--------+----------+-----------------+


In [30]:
eff_data18_JF17.save()

## Evaluate mc16 boosted:

In [31]:
mc16_df = pd.concat( (load_hdf(dpath_mc16), load_hdf(tpath_mc16)), axis=1)

In [32]:
emulate(mc16_df)

Emulate...: 100%|██████████| 4/4 [00:00<00:00, 38.22it/s]


In [33]:
eff_mc16 = Efficiency( output_path+'/mc16_boosted_efficiency_v9.root')

2022-03-13 10:20:33,244 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/mc16_boosted_efficiency_v9.root


In [34]:
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v9_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v9_ivarloose' , pidname='el_lhtight' , reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v9_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v9'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [00:00<00:00, 15.53it/s]


HLT_e17_lhvloose_nod0_ringer_v9_L1EM15VHI
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.7303  | 78407/78619  |
| L2Calo | 95.5939  | 75155/78619  |
|   L2   | 95.5838  | 75147/78619  |
| EFCalo | 75.3584  | 59246/78619  |
|  HLT   | 74.3217  | 58431/78619  |
+--------+----------+--------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 17.75it/s]


HLT_e28_lhtight_nod0_ringer_v9_ivarloose
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.8427  | 72990/73105  |
| L2Calo | 76.6678  | 56048/73105  |
|   L2   | 76.6623  | 56044/73105  |
| EFCalo | 62.6783  | 45821/73105  |
|  HLT   | 60.3926  | 44150/73105  |
+--------+----------+--------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 17.35it/s]


HLT_e60_lhmedium_nod0_ringer_v9_L1EM24VHI
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9635  | 73911/73938  |
| L2Calo | 79.7871  | 58993/73938  |
|   L2   | 79.7817  | 58989/73938  |
| EFCalo | 64.4608  | 47661/73938  |
|  HLT   | 62.9149  | 46518/73938  |
+--------+----------+--------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 17.20it/s]

HLT_e140_lhloose_nod0_ringer_v9
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9882  | 67947/67955  |
| L2Calo | 94.3919  | 64144/67955  |
|   L2   | 94.3875  | 64141/67955  |
| EFCalo | 73.8709  | 50199/67955  |
|  HLT   | 72.4877  | 49259/67955  |
+--------+----------+--------------+


In [35]:
eff_mc16.save()